In [1]:
import gmpy2
import random
from gmpy2 import mpz,is_strong_prp,is_strong_lucas_prp
import math

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
letterMap={}
letterMap['a']=0

reverseMap={}
reverseMap[0]='a'

for i in range(1,26):
    letterMap[chr(97+i)]=i
    reverseMap[i]=chr(97+i)

for i in range(0,10):
    reverseMap[26+i]=str(i)
    letterMap[str(i)] = 26+i

In [4]:
def generateTwoStrongPrimes():
    numberOfPrimes = 2
    strongPrimes=[]
    while numberOfPrimes>0:     
        initial="1"
        count = 5
        for i in range(0,510):
            initial+=str(random.randint(0,1))
        initial+="1"
        first=int(initial,2)
        
        while not gmpy2.is_prime(first):
            first = gmpy2.next_prime(first)
        second = gmpy2.next_prime(first)
        while not gmpy2.is_prime(second):
            second = gmpy2.next_prime(second)
        while True:
            third = gmpy2.next_prime(second)
            while not gmpy2.is_prime(third):
                third = gmpy2.next_prime(third)        
            if (first+third)/2 < second and gmpy2.is_prime((second-1)//2):
                strongPrimes.append(second)
                numberOfPrimes-=1
                break
            else:
                count-=1
                if count==0:
                    break
                first = second
                second=third
    return strongPrimes

In [5]:
def encryption(text,e,n,base):
    blockSize=int(math.floor(gmpy2.log(n)/gmpy2.log(36)))
    flag=0
    a=0
    for i in range(len(text)-1,-1,-1):
        a += letterMap[text[i]]*pow(base,flag)
        flag+=1
    blockSize=int(math.floor(gmpy2.log(n)/gmpy2.log(36)))
    cipherText = gmpy2.powmod(a,e,n)
    print(cipherText)
    back = cipherText
    out = ""
    a = []
    for i in range(blockSize,-1,-1):
        a.append(math.floor(back/pow(base,i)))
        back = back % pow(base,i)
    
    cipherText1 = ""
    for i in a:
        cipherText1 +=reverseMap[i]
    return cipherText1

def decryption(text,d,n,base):
    flag=0
    a=0
    for i in range(len(text)-1,-1,-1):
        a += letterMap[text[i]]*pow(base,flag)
        flag+=1
        
    blockSize=int(math.floor(gmpy2.log(n)/gmpy2.log(36)))
    cipherText = gmpy2.powmod(a,d,n)
    back = cipherText
    out = ""
    a = []
    for i in range(blockSize-1,-1,-1):
        a.append(math.floor(back/pow(base,i)))
        back = back % pow(base,i)
    
    cipherText1 = ""
    for i in a:
        cipherText1 +=reverseMap[i]
    return cipherText1

In [15]:
def saveForUser(userId,CAn,CAPrivateKey,a,e1):
    
    strongPrimes = a
    p = strongPrimes[0]
    q = strongPrimes[1]
    n = p*q
    phin = (p-1)*(q-1)
    while True:
        initialNo=""
        for i in range(0,156):
            initialNo+=str(random.randint(1,9))

        e  = int(initialNo)
        if gmpy2.gcd(e,phin)==1:
            break
    d = gmpy2.invert(e,phin)   
    pubKeyEnc = encryption(str(e),CAPrivateKey,CAn,36)
    NEnc = encryption(str(n),CAPrivateKey,CAn,36)
    priKeyEnc = encryption(str(d),CAPrivateKey,CAn,36)
    pEnc = encryption(str(p),CAPrivateKey,CAn,36)
    qEnc = encryption(str(q),CAPrivateKey,CAn,36)
    
    toSaveInPrivateFile = str(priKeyEnc)+"|"+str(pEnc)+"|"+str(qEnc)+"|"+str(NEnc)
    toSaveInPublicFile = str(userId)+"|"+str(pubKeyEnc)+"|"+str(NEnc)


    with open("publicKeys.txt", "a") as myfile:
        myfile.write(toSaveInPublicFile+"|\n")
    with open("privateKeyFor"+str(userId)+".txt", "w") as myfile:
        myfile.write(toSaveInPrivateFile+"|\n")
    
    
def main(a):
#     userId = sys.argv[1]
    fp = open("CAPublicKey","r")
    publicKey = fp.readlines()
    fp.close()
    fp = open("CAPrivateKey","r")
    privateKey = fp.readlines()
    fp.close()
    n = int(publicKey[0].replace("\n",""))
    e = int(publicKey[1].replace("\n",""))
    d = int(privateKey[2].replace("\n",""))
    saveForUser(24,n,d,a,e)

In [16]:
if __name__=="__main__":
    a = generateTwoStrongPrimes()
    main(a)
    print("Public Key Stored in publickeys.txt and Private Key stored in privatekeyFor<userid>.txt")

Strong Prime
Strong Prime
[mpz(6934104632955403748419032131298128582832415824496313126094149042212831371553384108008618947384768133004705697093145978276582412045221759226579420225297703), mpz(8736315405928751627910675446139810019643941433734144178552332467482262889929584646801683583852821600388586287420504794932373403490898271442656538946818919)]
Original
291799688272377593185456153935999196297475745994133898153145357774191127736423771981443487988518811217317886551528233514359853549751712161854557542476749689
970947184163498524063933934417888292847695661644121801898611470320729738587215341779558397700293960347822647459152265042968454041432379052312251363728717913574165945448841877414220470161874790861832362785816047683192155699343323648509881548287947115609075050610467650795332719779134730877868836843109601423536
6366407916443968985071085255431156146559669074033361089798140908648122171581316960288743933635127872152095696990268572207316576566945430106456769414115231616508104102335031228